# Implementação ACO - Problema do Caixeiro Viajante

In [46]:
# Célula de imports

import numpy as np
import random 
import matplotlib.pyplot as plt
import pandas as pd

### Planilha com a distância de cada cidade

In [47]:
# Célula para ler o planilha com as distâncias
df = pd.read_csv('distancia_matrix.csv', header=None)

cities_distances = df

cities_distances

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000,66.000,103.000,58.000,151.000,210.000,231.000,188.000,170.000,182.000,63.000,170.000,187.000,218.000,111.000,82.000,134.000,218.000,248.000,135.000
1,66.000,0.000,118.000,138.000,55.000,83.000,245.000,236.000,78.000,171.000,96.000,121.000,244.000,57.000,60.000,212.000,101.000,214.000,194.000,247.000
2,103.000,118.000,0.000,242.000,190.000,239.000,80.000,68.000,248.000,128.000,71.000,131.000,68.000,182.000,193.000,158.000,225.000,99.000,189.000,86.000
3,58.000,138.000,242.000,0.000,104.000,186.000,206.000,56.000,172.000,178.000,204.000,197.000,185.000,236.000,79.000,65.000,123.000,53.000,227.000,194.000
4,151.000,55.000,190.000,104.000,0.000,215.000,181.000,236.000,166.000,207.000,53.000,82.000,205.000,200.000,234.000,244.000,58.000,152.000,244.000,249.000
5,210.000,83.000,239.000,186.000,215.000,0.000,142.000,121.000,156.000,70.000,81.000,64.000,163.000,131.000,222.000,176.000,58.000,148.000,186.000,89.000
6,231.000,245.000,80.000,206.000,181.000,142.000,0.000,133.000,222.000,227.000,79.000,133.000,220.000,88.000,59.000,127.000,143.000,225.000,125.000,246.000
7,188.000,236.000,68.000,56.000,236.000,121.000,133.000,0.000,143.000,175.000,241.000,164.000,212.000,92.000,110.000,113.000,182.000,53.000,56.000,99.000
8,170.000,78.000,248.000,172.000,166.000,156.000,222.000,143.000,0.000,100.000,55.000,130.000,113.000,211.000,162.000,79.000,152.000,245.000,164.000,239.000
9,182.000,171.000,128.000,178.000,207.000,70.000,227.000,175.000,100.000,0.000,233.000,153.000,235.000,238.000,239.000,238.000,180.000,219.000,167.000,237.000


### PARÂMETROS GLOBAIS

In [56]:
NUM_CITIES = len(df)
EVAPORATION_RATE = 0.5
PHEROMONE_RATE = 100
ALPHA = 2
BETA = 3
ELITISM_FACTOR = 2

### CONSTRUÇÃO DE CAMINHOS

Para a realização do algoritmo, é necessário definir, inicialmente, o caminho que cada formiga irá percorrer a partir de sua cidade inicial. O calcúlo da probabilidade da formiga visitar a cidade vizinha é dada pela seguinte fórmula:

$$
p_{ij}^k = 
\frac{
    (\tau_{ij})^\alpha \cdot (\eta_{ij})^\beta
}{
    \sum\limits_{l \in \mathcal{N}_i^k} (\tau_{il})^\alpha \cdot (\eta_{il})^\beta
}, \quad \text{se } j \in \mathcal{N}_i^k
$$

onde 

p^kij =  probabilidade da formiga k ir de i para j.

nij = 1/distância ij

tij = quantidade de feromônio na aresta (i,j)(i,j).

Nik = ​ é o conjunto de cidades ainda não visitadas pela formiga kk a partir de i.

α = controla a importância do feromônio.

β = controla a importância da heurística.

In [49]:
# Função que calcula a atratividade de cada aresta

def calculateAtrac(origin, pheMatrix, distMatrix, not_visited):

    atracDict = {}

    for destiny in not_visited:

        pheromonEdge = pow(pheMatrix[origin][destiny], ALPHA)
        distEdge =     pow((1/distMatrix[origin][destiny]), BETA) 

        atracDict[destiny] = pheromonEdge * distEdge
    
    return atracDict

In [50]:
# Algoritmo para encontrar o "melhor" caminho a partir de um ponto inicial
'''
Recebe como parâmetros:
- Cidade Inicial (int)
- Matriz com o valor de feromônios de cada aresta
- Matriz de distâncias
'''

def findPath(initialCity, pheMatrix, distMatrix):
    path = []
    not_visited = list(range(NUM_CITIES))

    # Informa que a cidade inicial ja foi visitada
    path.append(initialCity)
    not_visited.remove(initialCity)


    # Enquanto todas a cidades não forem visitadas
    while not_visited:
        probabilities_list = []

        # Calcula a atratividade de cada cidade a partir do ponto inicial
        atrac_dict = calculateAtrac(initialCity, pheMatrix, distMatrix, not_visited)

        sum_atrac = 0
        # Calcula a soma das atratividades
        sum_atrac = sum(atrac_dict.values())

        # Calcula a probabilidade para cada cidade
        for city in not_visited:
            # Lista que armazena a cidade destino + sua probabilidade
            prob_list = []

            # Calcula a probabilidade da cidade
            cityProb = atrac_dict[city]/sum_atrac
            # Salva valores
            prob_list.append(city)
            prob_list.append(cityProb)

            # Salva tupla na lista com todas as probabilidades
            probabilities_list.append(prob_list)

        # Utiliza método de sorteio proporcional para selecionar o prox caminho
        cities, probs = zip(*probabilities_list)
        next_city = random.choices(cities, weights=probs, k=1)[0]

        # Adiciona essa cidade no caminho final
        path.append(next_city)
        # Marca a cidade como visitada
        not_visited.remove(next_city)
        # Define a cidade como sendo a inicial
        initialCity = next_city

    # Retorna o caminho final
    return path   
        



In [51]:
# Célula para a realização de testes

# Cria uma matriz com valores de feromonios igual a 1
pheMatrix = np.ones((NUM_CITIES, NUM_CITIES))

# Lista para salvar os caminhos
paths = []

# Calcula caminhos partindo de cada cidade
for i in range(NUM_CITIES):
    path = findPath(i, pheMatrix, cities_distances)
    paths.append(path)
    print(f'Caminho cidade [{i}]: {path}')

Caminho cidade [0]: [0, 10, 14, 16, 4, 1, 5, 11, 18, 15, 8, 9, 2, 12, 19, 7, 17, 3, 13, 6]
Caminho cidade [1]: [1, 16, 5, 9, 10, 11, 19, 6, 14, 15, 2, 7, 18, 17, 3, 0, 4, 8, 12, 13]
Caminho cidade [2]: [2, 6, 18, 16, 5, 10, 4, 3, 13, 1, 0, 8, 15, 9, 14, 11, 19, 12, 17, 7]
Caminho cidade [3]: [3, 7, 17, 13, 5, 9, 11, 16, 2, 12, 19, 10, 4, 1, 14, 6, 18, 15, 8, 0]
Caminho cidade [4]: [4, 16, 1, 5, 11, 19, 12, 2, 6, 14, 17, 10, 8, 15, 13, 7, 3, 0, 9, 18]
Caminho cidade [5]: [5, 8, 1, 4, 10, 13, 7, 18, 14, 6, 2, 19, 11, 9, 0, 3, 15, 17, 16, 12]
Caminho cidade [6]: [6, 15, 14, 16, 5, 9, 2, 10, 0, 3, 17, 7, 19, 11, 8, 1, 4, 12, 18, 13]
Caminho cidade [7]: [7, 6, 14, 16, 11, 10, 2, 17, 3, 4, 1, 12, 8, 9, 5, 19, 18, 15, 0, 13]
Caminho cidade [8]: [8, 12, 2, 18, 15, 3, 7, 17, 10, 5, 9, 11, 14, 0, 1, 13, 6, 16, 4, 19]
Caminho cidade [9]: [9, 5, 8, 10, 17, 7, 2, 12, 19, 11, 4, 16, 14, 6, 3, 0, 15, 18, 1, 13]
Caminho cidade [10]: [10, 2, 19, 5, 8, 18, 7, 14, 0, 11, 4, 1, 12, 13, 6, 15, 3, 17, 9, 16

### ATUALIZAÇÃO DO FEROMÔNIO

Após realizado o cálculo dos caminhos, é necessário atualizar o feromônio associado a cada aresta que foi visitada pelas formigas. A atualização se dá pela fórmula:

$$
\tau_{ij}(t + 1) = (1 - \rho)\tau_{ij}(t) + \sum_{k=1}^{m} \Delta \tau_{ij}^k(t)
$$

Onde:

Tij(t) = quantidade de feromônio na aresta (i, j) no tempo t

p =  taxa de evaporação (entre 0 e 1)

deltaTkij(t) =  quantidade de feromônio depositada pela formiga kk na 
aresta (i, j)

m = numero de formigas

O valor a ser depositado será dado por:

$$
\Delta \tau_{ij}^k =
\begin{cases}
\frac{Q}{L_k}, & \text{se a aresta } (i,j) \in S_k \\
0, & \text{caso contrário}
\end{cases}
$$

Onde:

Q = constante de depósito de feromônio

Lk​ = comprimento total da rota percorrida pela formiga kk

Sk​ = rota construída pela formiga k



Logo, precisamos calcular Lk para cada rota encontrada

In [52]:
# Função para calcular o comprimento de cada rota

def calculatePathLength(paths, distMatrix):
    pathsLengths = []

    # Itera por cada caminho gerado
    for path in paths:
        # Pega o indice da cidade i e pega sua aresta com i + 1
        length = 0
        pathLength = []
        for i in range(len(path) -1):

            origin = path[i]
            destiny = path[i + 1]
            # Adiciona na distancia final
            length += distMatrix[origin][destiny]

        # Monta a tupla com caminho e length
        pathLength.append(path)
        pathLength.append(length)

        # Adiciona na lista final
        pathsLengths.append(pathLength)

    return pathsLengths


In [53]:
# Célula para a realização de testes

pathsLengths = calculatePathLength(paths, cities_distances)

for i, path in enumerate(pathsLengths):
    print(f'Caminho cidade [{i}]: {path[0]}\nDistancia Total: {path[1]}\n')


Caminho cidade [0]: [0, 10, 14, 16, 4, 1, 5, 11, 18, 15, 8, 9, 2, 12, 19, 7, 17, 3, 13, 6]
Distancia Total: 1754.0

Caminho cidade [1]: [1, 16, 5, 9, 10, 11, 19, 6, 14, 15, 2, 7, 18, 17, 3, 0, 4, 8, 12, 13]
Distancia Total: 2034.0

Caminho cidade [2]: [2, 6, 18, 16, 5, 10, 4, 3, 13, 1, 0, 8, 15, 9, 14, 11, 19, 12, 17, 7]
Distancia Total: 2223.0

Caminho cidade [3]: [3, 7, 17, 13, 5, 9, 11, 16, 2, 12, 19, 10, 4, 1, 14, 6, 18, 15, 8, 0]
Distancia Total: 1916.0

Caminho cidade [4]: [4, 16, 1, 5, 11, 19, 12, 2, 6, 14, 17, 10, 8, 15, 13, 7, 3, 0, 9, 18]
Distancia Total: 1631.0

Caminho cidade [5]: [5, 8, 1, 4, 10, 13, 7, 18, 14, 6, 2, 19, 11, 9, 0, 3, 15, 17, 16, 12]
Distancia Total: 1846.0

Caminho cidade [6]: [6, 15, 14, 16, 5, 9, 2, 10, 0, 3, 17, 7, 19, 11, 8, 1, 4, 12, 18, 13]
Distancia Total: 1927.0

Caminho cidade [7]: [7, 6, 14, 16, 11, 10, 2, 17, 3, 4, 1, 12, 8, 9, 5, 19, 18, 15, 0, 13]
Distancia Total: 1917.0

Caminho cidade [8]: [8, 12, 2, 18, 15, 3, 7, 17, 10, 5, 9, 11, 14, 0, 1,

#### Elitismo

Para melhorar a performance e a qualidade das soluções, pode-se utilizar elitismo, o qual fornece um reforço adicional nas arestas pertecentes ao melhor percurso achado desde o inicio do algoritmo. O melhor caminho encontrado é representado por TijBS

$$
\tau_{ij} = (1 - \rho)\tau_{ij} + \sum_{k=1}^{m} \Delta \tau_{ij}^{(k)} + e \cdot \Delta \tau_{ij}^{(bs)}
$$

onde:

$$
\Delta \tau_{ij}^{(bs)} =
\begin{cases}
\frac{Q}{L_{bs}}, & \text{se a aresta } (i, j) \text{ pertence ao percurso } T_{bs} \\
0, & \text{caso contrário}
\end{cases}
$$

In [57]:
def elitismDeposit(delta_pheromone, bestPathLength):

    # Sepera os valores de comprimento e caminho
    best_path, best_length = bestPathLength

    # Deposita o feromônio extra no melhor caminho
    for i in range(len(best_path) - 1):
        origin = best_path[i]
        destiny = best_path[i + 1]

        # calcula o feromonio extra
        extra_pheromone = ELITISM_FACTOR * (PHEROMONE_RATE/best_length)
        # Adiciona o feromonio extra
        delta_pheromone[origin][destiny] += extra_pheromone
        delta_pheromone[destiny][origin] += extra_pheromone

    return delta_pheromone


Realização do depósito e evaporação de feromônio

In [ ]:
# Função para depositar os feromônios e 
# retornar a matriz de feromônios atualizada

def updatePheromone(pathsLengths, pheMatrix, bestPathLength):

    # Cria uma matriz provisoria para armazenar a soma do de feromonios
    delta_pheromone = [[0.0 for _ in range(NUM_CITIES)] for _ in range(NUM_CITIES)]

    # Itera em cada caminho, adicionando os feromonios em cada aresta passada
    for pathLength in pathsLengths:
        # Separa os valores do caminho e o length
        path, length = pathLength

        # Realiza o depósito na matriz provisoria
        for i in range(len(path) -1):
            # Pega incio e fim para definir qual aresta será depositada
            origin = path[i]
            destiny = path[i + 1]

            # Adiciona tanto na aresta da ida quanto na volta, uma vez que não é um grafo orientado
            delta_pheromone[origin][destiny] += PHEROMONE_RATE/length
            delta_pheromone[destiny][origin] += PHEROMONE_RATE/length
        
    # Realiza o depósito do elitismo na melhor aresta
    delta_pheromone = elitismDeposit(delta_pheromone,bestPathLength)

    # Itera pela matriz final realizando os métodos de evaporação e depósito
    for i in range(NUM_CITIES):
        for j in range(NUM_CITIES):

            # EVAPORAÇÃO 
            pheMatrix[i][j] *= (1 - EVAPORATION_RATE)
            # DEPÓSITO
            pheMatrix[i][j] += delta_pheromone[i][j]

    return pheMatrix


In [55]:
# Célula para a realização de testes

# Imprime a matriz de feromônios atualizada com pandas
updated_phe = []
updated_phe = updatePheromone(pathsLengths, pheMatrix)

df = pd.DataFrame(updated_phe)
pd.options.display.float_format = "{:,.3f}".format

print("MATRIZ DE FEROMÔNIOS APÓS A PRIMEIRA ITERAÇÃO")
df


MATRIZ DE FEROMÔNIOS APÓS A PRIMEIRA ITERAÇÃO


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.500,0.656,0.500,1.080,0.658,0.557,0.500,0.500,0.597,0.615,0.868,0.600,0.500,0.606,0.652,0.741,0.500,0.500,0.500,0.500
1,0.656,0.500,0.500,0.500,1.111,0.618,0.500,0.500,0.669,0.500,0.500,0.500,0.654,0.961,0.685,0.500,0.720,0.500,0.616,0.500
2,0.500,0.500,0.500,0.500,0.500,0.500,0.834,0.800,0.500,0.788,0.704,0.500,0.912,0.500,0.500,0.549,0.610,0.606,0.553,0.777
3,1.080,0.500,0.500,0.500,0.597,0.556,0.562,0.910,0.500,0.554,0.500,0.500,0.500,0.602,0.500,0.716,0.554,1.057,0.500,0.500
4,0.658,1.111,0.500,0.597,0.500,0.500,0.626,0.500,0.549,0.500,0.890,0.607,0.552,0.500,0.500,0.500,0.981,0.554,0.500,0.553
5,0.557,0.618,0.500,0.556,0.500,0.500,0.558,0.500,0.662,1.064,0.652,0.920,0.500,0.552,0.500,0.500,0.826,0.500,0.500,0.722
6,0.500,0.500,0.834,0.562,0.626,0.558,0.500,0.552,0.500,0.500,0.500,0.570,0.500,0.656,1.181,0.597,0.609,0.500,0.784,0.549
7,0.500,0.500,0.800,0.910,0.500,0.500,0.552,0.500,0.500,0.500,0.500,0.500,0.500,0.671,0.545,0.554,0.500,1.067,0.880,0.663
8,0.597,0.669,0.500,0.500,0.549,0.662,0.500,0.500,0.500,0.911,0.915,0.606,0.654,0.500,0.500,0.892,0.563,0.500,0.545,0.500
9,0.615,0.500,0.788,0.554,0.500,1.064,0.500,0.500,0.911,0.500,0.549,0.715,0.558,0.500,0.545,0.545,0.545,0.545,0.624,0.500
